In [5]:
# from squeezenet_model import SqueezeNet

In [4]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt

# Define paths to your dataset
train_data_dir = './train/'
test_data_dir = './test/'
val_data_dir = './val/'

# Set parameters
img_width, img_height = 224, 224
batch_size = 32
epochs = 10

# Create data generators
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'  # assuming binary classification (drowsy and non-drowsy)
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

# Load SqueezeNet (use a custom implementation or one from TensorFlow Hub)
# For example, using a function `SqueezeNet()` that returns a SqueezeNet model
model = SqueezeNet(input_shape=(224, 224, 3), out_classes=2, lr_rate=0.0001, train=True)
x = model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=10)

# Plotting
plt.figure(1)

# Accuracy plot
plt.subplot(211)
plt.plot(history.history['accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')

# Loss plot
plt.subplot(212)
plt.plot(history.history['loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')

plt.tight_layout()
plt.show()


Found 2149 images belonging to 2 classes.
Found 272 images belonging to 2 classes.
Found 268 images belonging to 2 classes.


AttributeError: module 'tensorflow' has no attribute 'placeholder'

In [ ]:
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Dropout, GlobalAveragePooling2D, Dense

def fire_module(x, squeeze_planes, expand_planes):
    """
    Implements the fire module
    """
    # Squeeze layer
    x = Conv2D(squeeze_planes, (1, 1), activation='relu', padding='valid')(x)
    
    # Expand layer
    left = Conv2D(expand_planes, (1, 1), activation='relu', padding='valid')(x)
    right = Conv2D(expand_planes, (3, 3), activation='relu', padding='same')(x)
    
    # Concatenate along the channel dimension
    x = concatenate([left, right], axis=-1)
    return x

def SqueezeNet(input_shape=(224, 224, 3), classes=1000):
    """
    Creates the SqueezeNet architecture
    """
    inputs = Input(shape=input_shape)
    
    # Initial convolution layer
    x = Conv2D(96, (7, 7), strides=(2, 2), activation='relu', padding='same')(inputs)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid')(x)
    
    # Fire modules
    x = fire_module(x, 16, 64)
    x = fire_module(x, 16, 64)
    x = fire_module(x, 32, 128)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid')(x)
    x = fire_module(x, 32, 128)
    x = fire_module(x, 48, 192)
    x = fire_module(x, 48, 192)
    x = fire_module(x, 64, 256)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid')(x)
    x = fire_module(x, 64, 256)

    # Final layers
    x = Dropout(0.5)(x)
    x = Conv2D(classes, (1, 1), padding='valid', activation='relu')(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(classes, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=x)
    return model

# Create the model
model = SqueezeNet(input_shape=(224, 224, 3), classes=1000)
model.summary()


In [6]:
import tensorflow as tf
from keras.layers import Input, Conv2D, MaxPooling2D, Concatenate, Dropout, Flatten, Dense
from keras.models import Model

def fire_module(x, squeeze=16, expand=64):
    x = Conv2D(squeeze, (1, 1), padding='valid', activation='relu')(x)
    left = Conv2D(expand, (1, 1), padding='valid', activation='relu')(x)
    right = Conv2D(expand, (3, 3), padding='same', activation='relu')(x)
    x = Concatenate()([left, right])
    return x


In [7]:
def SqueezeNet(input_shape=(224, 224, 3), classes=1000):
    inputs = Input(shape=input_shape)

    # Initial convolution layer
    x = Conv2D(64, (3, 3), strides=(2, 2), padding='same', activation='relu')(inputs)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)

    # Fire modules
    x = fire_module(x, squeeze=16, expand=64)
    x = fire_module(x, squeeze=16, expand=64)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)

    x = fire_module(x, squeeze=32, expand=128)
    x = fire_module(x, squeeze=32, expand=128)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)

    x = fire_module(x, squeeze=48, expand=192)
    x = fire_module(x, squeeze=48, expand=192)
    x = fire_module(x, squeeze=64, expand=256)
    x = fire_module(x, squeeze=64, expand=256)

    # Final convolution layer
    x = Conv2D(classes, (1, 1), padding='valid', activation='relu')(x)
    x = Flatten()(x)
    outputs = Dense(classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model


In [12]:
model = SqueezeNet(input_shape=(224, 224, 3), classes=1)  # For binary classification, use classes=2
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# You can now train the model using your data
# model.fit(train_data, train_labels, batch_size=32, epochs=10, validation_data=(val_data, val_labels))

In [13]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_26 (Conv2D)          (None, 112, 112, 64)         1792      ['input_2[0][0]']             
                                                                                                  
 max_pooling2d_3 (MaxPoolin  (None, 55, 55, 64)           0         ['conv2d_26[0][0]']           
 g2D)                                                                                             
                                                                                                  
 conv2d_27 (Conv2D)          (None, 55, 55, 16)           1040      ['max_pooling2d_3[0][0]'

In [16]:
# Define paths to your dataset
train_data_dir = './train/'
test_data_dir = './test/'
val_data_dir = './val/'

# Set parameters
img_width, img_height = 224, 224
batch_size = 64
epochs = 10

# Create data generators
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'  # assuming binary classification (drowsy and non-drowsy)
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 2149 images belonging to 2 classes.
Found 272 images belonging to 2 classes.
Found 268 images belonging to 2 classes.


In [17]:
history = model.fit(train_generator, epochs=epochs, validation_data=val_generator)

Epoch 1/10
34/34 [==============================] - 67s 2s/step - loss: nan - accuracy: 0.4658 - val_loss: nan - val_accuracy: 0.4664
Epoch 2/10
34/34 [==============================] - 65s 2s/step - loss: nan - accuracy: 0.4658 - val_loss: nan - val_accuracy: 0.4664
Epoch 3/10
14/34 [===========>..................] - ETA: 36s - loss: nan - accuracy: 0.4580

KeyboardInterrupt: 